In [1]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
### Ваш код ###
from sklearn.model_selection import train_test_split

target = data['MEDV']

source = data.drop('MEDV', axis=1)

train_data, test_data, train_labels, test_labels = train_test_split(source, target, test_size=0.2)
test_data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
416,10.83420,0.0,18.10,0.0,0.679,6.782,90.8,1.8195,24.0,666.0,20.2,21.57,25.79
360,4.54192,0.0,18.10,0.0,0.770,6.398,88.0,2.5182,24.0,666.0,20.2,374.56,7.79
99,0.06860,0.0,2.89,0.0,0.445,7.416,62.5,3.4952,2.0,276.0,18.0,396.90,6.19
457,8.20058,0.0,18.10,0.0,0.713,5.936,80.3,2.7792,24.0,666.0,20.2,3.50,16.94
349,0.02899,40.0,1.25,0.0,0.429,6.939,34.5,8.7921,1.0,335.0,19.7,389.85,5.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,0.06724,0.0,3.24,0.0,0.460,6.333,17.2,5.2146,4.0,430.0,16.9,375.21,7.34
308,0.49298,0.0,9.90,0.0,0.544,6.635,82.5,3.3175,4.0,304.0,18.4,396.90,4.54
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
397,7.67202,0.0,18.10,0.0,0.693,5.747,98.9,1.6334,24.0,666.0,20.2,393.10,19.92


2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [6]:
### Ваш код ###
r2_lr = 0
r2_ridge = 0
r2_lasso = 0
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

# Обучение стандартной регрессии
lr = LinearRegression()
lr.fit(train_data, train_labels)
lr_pred = lr.predict(test_data)
r2_lr = r2_score(test_labels, lr_pred)

# Обучение Ridge регрессии
ridge = Ridge()
ridge.fit(train_data, train_labels)
ridge_pred = ridge.predict(test_data)
r2_ridge = r2_score(test_labels, ridge_pred)

# Обучение Lasso регрессии
lasso = Lasso()
lasso.fit(train_data, train_labels)
lasso_pred = lasso.predict(test_data)
r2_lasso = r2_score(test_labels, lasso_pred)

# Вывод R2 на тестовой выборке

results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

display(results_regression)

# Вывод таблицы весов параметров для каждой модели
dataset_fields = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

models_weights = pd.DataFrame(columns = ['model'].extend(dataset_fields))

results = {
    'lr': {'model':'lr'} | dict(zip(dataset_fields, lr.coef_)),
    'ridge': {'model':'ridge'} | dict(zip(dataset_fields, ridge.coef_)),
    'lasso': {'model':'lasso'} | dict(zip(dataset_fields, lasso.coef_))
}

for model in ['lr', 'ridge', 'lasso']:
    df_dictionary = pd.DataFrame([results[model]])
    models_weights = pd.concat([models_weights, df_dictionary], ignore_index=True)

display(models_weights)



,model,task,R2
0,LR,task2,0.692877
1,Ridge,task2,0.684244
2,Lasso,task2,0.668904


,model,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,lr,-0.108044,0.037411,0.010737,2.548323,-15.312397,4.222723,-0.014212,-1.533795,0.294009,-0.014317,-0.912030,0.007349,-0.460430
1,ridge,-0.104939,0.039004,-0.014960,2.461600,-8.182171,4.229461,-0.019566,-1.421634,0.280102,-0.015082,-0.840959,0.007547,-0.472996
2,lasso,-0.072493,0.041534,-0.000000,0.000000,-0.000000,1.416464,0.013874,-0.756944,0.270010,-0.017509,-0.701697,0.006729,-0.698383


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [16]:


### Ваш код ###

r2_ridge_grid_search = 0
r2_ridge_cv = 0
r2_lasso_grid_search = 0
r2_lasso_cv = 0

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import RidgeCV, LassoCV

alpha = np.power(10, np.arange(-5., 5.))

# Ridge GridSearcgCV
param_grid = {'alpha': alpha}
grid_search_ridge = GridSearchCV(Ridge(), param_grid, cv=KFold(n_splits=5))
grid_search_ridge.fit(train_data, train_labels)
ridge_best_alpha = grid_search_ridge.best_params_['alpha']
ridge_best = Ridge(alpha=ridge_best_alpha)
ridge_best.fit(train_data, train_labels)
ridge_best_pred = ridge_best.predict(test_data)
r2_ridge_grid_search = r2_score(test_labels, ridge_best_pred)

# Lasso GridSearcgCV
grid_search_lasso = GridSearchCV(Lasso(), param_grid, cv=KFold(n_splits=5))
grid_search_lasso.fit(train_data, train_labels)
lasso_best_alpha = grid_search_lasso.best_params_['alpha']
lasso_best = Lasso(alpha=lasso_best_alpha)
lasso_best.fit(train_data, train_labels)
lasso_best_pred = lasso_best.predict(test_data)
r2_lasso_grid_search = r2_score(test_labels, lasso_best_pred)

# RidgeCV
ridge_cv = RidgeCV(alphas=alpha)
ridge_cv.fit(train_data, train_labels)
ridge_cv_pred = ridge_cv.predict(test_data)
r2_ridge_cv = r2_score(test_labels, ridge_cv_pred)

# LassoCV
lasso_cv = LassoCV(alphas=alpha)
lasso_cv.fit(train_data, train_labels)
lasso_cv_pred = lasso_cv.predict(test_data)
r2_lasso_cv = r2_score(test_labels, lasso_cv_pred)


results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]
results_regression

,model,task,R2
0,LR,task2,0.692877
1,Ridge,task2,0.684244
2,Lasso,task2,0.668904
3,Ridge_GridSearchCV,task3,0.692877
4,RidgeCV,task3,0.691734
5,Lasso_GridSearchCV,task3,0.692874
6,LassoCV,task3,0.692874


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [17]:


### Ваш код ###

r2_ridge_standart_scaler = 0
r2_ridge_min_max_scaler = 0
r2_lasso_standart_scaler = 0
r2_lasso_min_max_scaler = 0

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# pipeline with StandardScaler
pipe_ridge_std = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge())
])

# pipeline with MinMaxScaler
pipe_ridge_mm = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regressor', Ridge())
])

# fit the pipelines
pipe_ridge_std.fit(train_data, train_labels)
pipe_ridge_mm.fit(train_data, train_labels)

# calculate R2 score on the test data
r2_ridge_standart_scaler = r2_score(train_labels, pipe_ridge_std.predict(train_data))
r2_ridge_min_max_scaler = r2_score(test_labels, pipe_ridge_mm.predict(test_data))


# то же для Lasso
pipe_lasso_std = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Lasso())
])

pipe_lasso_mm = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regressor', Lasso())
])

pipe_lasso_std.fit(train_data, train_labels)
pipe_lasso_mm.fit(train_data, train_labels)

r2_lasso_standart_scaler = r2_score(train_labels, pipe_lasso_std.predict(train_data))
r2_lasso_min_max_scaler = r2_score(test_labels, pipe_lasso_mm.predict(test_data))


results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

results_regression

,model,task,R2
0,LR,task2,0.692877
1,Ridge,task2,0.684244
2,Lasso,task2,0.668904
3,Ridge_GridSearchCV,task3,0.692877
4,RidgeCV,task3,0.691734
5,Lasso_GridSearchCV,task3,0.692874
6,LassoCV,task3,0.692874
7,Ridge_StandardScaler,task4,0.747686
8,Ridge_MinMaxScaler,task4,0.690004
9,Lasso_StandardScaler,task4,0.673724


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [22]:


### Ваш код ###

r2_ridge_standart_scaler_cv = 0
r2_ridge_min_max_scaler_cv = 0
r2_lasso_standart_scaler_cv = 0
r2_lasso_min_max_scaler_cv = 0

alpha_grid = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

scaler = StandardScaler()
scaler.fit_transform(train_data)
train_scaled_data = scaler.transform(train_data)
test_scaled_data = scaler.transform(test_data)

r2_ridge_standart_scaler_cv = r2_score(train_labels, pipe_ridge_std.predict(train_scaled_data))
r2_ridge_min_max_scaler_cv = r2_score(test_labels, pipe_ridge_mm.predict(test_scaled_data))
r2_lasso_standart_scaler_cv = r2_score(train_labels, pipe_lasso_std.predict(train_scaled_data))
r2_lasso_min_max_scaler_cv = r2_score(test_labels, pipe_lasso_mm.predict(test_scaled_data))

results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

results_regression

c:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


,model,task,R2
0,LR,task2,0.692877
1,Ridge,task2,0.684244
2,Lasso,task2,0.668904
3,Ridge_GridSearchCV,task3,0.692877
4,RidgeCV,task3,0.691734
5,Lasso_GridSearchCV,task3,0.692874
6,LassoCV,task3,0.692874
7,Ridge_StandardScaler,task4,0.747686
8,Ridge_MinMaxScaler,task4,0.690004
9,Lasso_StandardScaler,task4,0.673724


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [ ]:


### Ваш код ###

r2_ridge_standart_scaler_poly = 0
r2_ridge_min_max_scaler_poly = 0
r2_lasso_standart_scaler_poly = 0
r2_lasso_min_max_scaler_poly = 0
results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [ ]:


### Ваш код ###

r2_ridge_standart_scaler_poly_cv = 0
r2_ridge_min_max_scaler_poly_cv = 0
r2_lasso_standart_scaler_poly_cv = 0
r2_lasso_min_max_scaler_poly_cv = 0
results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
r2_best_model = 0
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

In [ ]:
results_regression

http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
data = pd.read_csv('adult.csv')
data.head()

9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [ ]:


### Ваш код ###

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [ ]:


### Ваш код ###

f1_most_frequent = 0
acc_most_frequent = 0
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [ ]:


### Ваш код ###

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [ ]:
tasks+=1

### Ваш код ###

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [ ]:


### Ваш код ###

f1_LR_del_missings = 0
acc_LR_del_missings = 0
f1_KNN_del_missings = 0
acc_KNN_del_missings = 0
f1_SVM_del_missings = 0
acc_SVM_del_missings = 0
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [ ]:


### Ваш код ###

f1_RF = 0
acc_RF = 0
f1_GB = 0
acc_GB = 0
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [ ]:
results_classification